Import the required libraries

In [1]:
from d3rlpy.datasets import get_cartpole
from d3rlpy.algos import DiscreteCQL, DQN
from d3rlpy.metrics.scorer import discounted_sum_of_advantage_scorer
from d3rlpy.metrics.scorer import evaluate_on_environment
from d3rlpy.dataset import Episode
from d3rlpy.dataset import MDPDataset

from d3rlpy.metrics.scorer import td_error_scorer
from d3rlpy.metrics.scorer import average_value_estimation_scorer
from sklearn.model_selection import train_test_split

import import_ipynb
import numpy as np
from random import random
from create_dataset import CreateDataset
from FootballEnv import FootballEnv

importing Jupyter notebook from FootballEnv.ipynb


Helper function to create a dummy dataset

In [2]:
def create_dataset():

    dataset_maker = CreateDataset()
    dataset_maker.loadFile('data.json')

    observations, actions, rewards = dataset_maker.createEpisodeDataset()
    terminals = np.array([[0, 0, 0, 0, 1] for i in range(len(observations))])

    return MDPDataset(
        observations,
        actions,
        rewards, 
        terminals,
    )

In [3]:
dataset = create_dataset()
train_episodes, test_episodes = train_test_split(dataset, test_size=0.2)


In [4]:
# setup CQL algorithm
cql = DiscreteCQL(use_gpu=False)

env = FootballEnv()

# start training
output = cql.fit(

    train_episodes,
    eval_episodes=test_episodes,
    n_epochs=50,
    scorers={
        'environment': evaluate_on_environment(env), # evaluate with Football Env
        'advantage': discounted_sum_of_advantage_scorer, # smaller is better
        'td_error': td_error_scorer, # smaller is better
        'value_scale': average_value_estimation_scorer # smaller is better
    }
    
)

2022-02-09 18:59.37 [debug    ] RoundIterator is selected.
2022-02-09 18:59.37 [info     ] Directory is created at d3rlpy_logs\DiscreteCQL_20220209185937
2022-02-09 18:59.37 [debug    ] Building models...
2022-02-09 18:59.37 [debug    ] Models have been built.
2022-02-09 18:59.37 [info     ] Parameters are saved to d3rlpy_logs\DiscreteCQL_20220209185937\params.json params={'action_scaler': None, 'alpha': 1.0, 'batch_size': 32, 'encoder_factory': {'type': 'default', 'params': {'activation': 'relu', 'use_batch_norm': False, 'dropout_rate': None}}, 'gamma': 0.99, 'generated_maxlen': 100000, 'learning_rate': 6.25e-05, 'n_critics': 1, 'n_frames': 1, 'n_steps': 1, 'optim_factory': {'optim_cls': 'Adam', 'betas': (0.9, 0.999), 'eps': 1e-08, 'weight_decay': 0, 'amsgrad': False}, 'q_func_factory': {'type': 'mean', 'params': {'bootstrap': False, 'share_encoder': False}}, 'real_ratio': 1.0, 'reward_scaler': None, 'scaler': None, 'target_reduction_type': 'min', 'target_update_interval': 8000, 'use_

Epoch 1/50: 100%|██████████| 5/5 [00:00<00:00, 74.63it/s, loss=1.59]


2022-02-09 18:59.37 [info     ] DiscreteCQL_20220209185937: epoch=1 step=5 epoch=1 metrics={'time_sample_batch': 0.00020012855529785156, 'time_algorithm_update': 0.012398958206176758, 'loss': 1.5351174354553223, 'time_step': 0.01319890022277832, 'environment': 2.6485580262271236, 'advantage': -0.2708004844428999, 'td_error': 0.08050351134640248, 'value_scale': 0.062185308003487684} step=5
2022-02-09 18:59.37 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220209185937\model_5.pt


Epoch 2/50: 100%|██████████| 5/5 [00:00<00:00, 83.33it/s, loss=1.49]


2022-02-09 18:59.37 [info     ] DiscreteCQL_20220209185937: epoch=2 step=10 epoch=2 metrics={'time_sample_batch': 0.00040044784545898435, 'time_algorithm_update': 0.011199092864990235, 'loss': 1.472410249710083, 'time_step': 0.012000370025634765, 'environment': 2.6480849834463083, 'advantage': -0.07488980034329944, 'td_error': 0.08331471621612756, 'value_scale': 0.043040832543435194} step=10
2022-02-09 18:59.37 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220209185937\model_10.pt


...

2022-02-09 18:59.56 [info     ] DiscreteCQL_20220209185937: epoch=48 step=240 epoch=48 metrics={'time_sample_batch': 0.0005999088287353515, 'time_algorithm_update': 0.0160006046295166, 'loss': 0.6709317147731781, 'time_step': 0.017200326919555663, 'environment': 2.4758073565326795, 'advantage': -0.7155714972054082, 'td_error': 0.6166489564885053, 'value_scale': -0.060230982451078795} step=240
2022-02-09 18:59.56 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220209185937\model_240.pt


Epoch 49/50: 100%|██████████| 5/5 [00:00<00:00, 62.50it/s, loss=0.885]


2022-02-09 18:59.56 [info     ] DiscreteCQL_20220209185937: epoch=49 step=245 epoch=49 metrics={'time_sample_batch': 0.0, 'time_algorithm_update': 0.015000009536743164, 'loss': 0.6932828068733216, 'time_step': 0.015600252151489257, 'environment': 2.2020624742118207, 'advantage': -0.7207827015588982, 'td_error': 0.6257488033530029, 'value_scale': -0.060410601630186044} step=245
2022-02-09 18:59.57 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220209185937\model_245.pt


Epoch 50/50: 100%|██████████| 5/5 [00:00<00:00, 57.47it/s, loss=0.552]


2022-02-09 18:59.57 [info     ] DiscreteCQL_20220209185937: epoch=50 step=250 epoch=50 metrics={'time_sample_batch': 0.0008016586303710938, 'time_algorithm_update': 0.015799236297607423, 'loss': 0.6844325304031372, 'time_step': 0.017000198364257812, 'environment': 2.0041963843238486, 'advantage': -0.7317800378155631, 'td_error': 0.6386047639419262, 'value_scale': -0.06021358259022236} step=250
2022-02-09 18:59.57 [info     ] Model parameters are saved to d3rlpy_logs\DiscreteCQL_20220209185937\model_250.pt


In [5]:
obs = env.reset()

ds = CreateDataset()
ds.loadFile('data.json')

observations, actions, rewards = ds.createEpisodeDataset()
answers = {}
for situation in observations:

    situation = np.array([ds.PASS, ds.CARRY, ds.CARRY, ds.CARRY, ds.CARRY], dtype=np.float32)
    predictions = cql.predict([situation])[0]

    p = ds.ID_to_str[predictions]
    if not (p in answers): answers[p] = 1
    else: answers[p] += 1

    # print(ds.ID_to_str[predictions])

In [6]:
counts = {}

for item in actions:
    if not (item in counts): counts[item] = 1
    else: counts[item] += 1

counts


{2.0: 105, 0.0: 168, 3.0: 7, 1.0: 1}